In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [16]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    data = pd.read_csv('university_towns.txt', sep="\n", header=None)
    data.columns = ["RegionName"]
    data['State'] = data['RegionName'].where(data['RegionName'].str.endswith('[edit]'))
    data["RegionName"]= data["RegionName"].str.replace('\[.*','').str.replace('\s*\(.*','')
    data["State"]= data["State"].str.replace('\[.*','').fillna(method='ffill')
    data = data[data['State']!=data['RegionName']]
    return data[['State','RegionName']]
get_list_of_university_towns().info()
get_list_of_university_towns().sample(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 1 to 566
Data columns (total 2 columns):
State         517 non-null object
RegionName    517 non-null object
dtypes: object(2)
memory usage: 12.1+ KB


,State,RegionName
247,Minnesota,St. Cloud
230,Michigan,Mount Pleasant
448,Pennsylvania,West Chester
160,Kentucky,Morehead
357,North Carolina,Winston-Salem
536,Washington,Bellingham
181,Maine,"Lewiston, Maine"
221,Michigan,Berrien Springs
167,Louisiana,Grambling
519,Vermont,Lyndonville


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    quarterly = pd.read_excel('gdplev.xls',header=5, skiprows=[6,7],parse_cols="E,G")
    quarterly = quarterly[quarterly.index.str.startswith('20')]
    start = ''
    row=1
    flag=0 # not in recession already
    while row < len(quarterly)-1:
        if (quarterly.iloc[row,0]<quarterly.iloc[row-1,0]) & (quarterly.iloc[row,0]>quarterly.iloc[row+1,0]) & (flag==0):
            start+=quarterly.index[row]
            row+=2 # 
            flag=1 # next consecutive decline doesn't count as beginning if flag==1, meaning recession begins
        else:
            row+=1
            flag=0 # not in recession
    return start
print(get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    quarterly = pd.read_excel('gdplev.xls',header=5, skiprows=[6,7],parse_cols="E,G")
    quarterly = quarterly[quarterly.index.str.startswith('20')]
    end = ''
    row=quarterly.index.get_loc(get_recession_start())+2
    while row < len(quarterly)-1:
        if (quarterly.iloc[row,0]>quarterly.iloc[row-1,0]) & (quarterly.iloc[row,0]<quarterly.iloc[row+1,0]):
            end+=quarterly.index[row+1]
            break
        else:
            row+=1
    return end
get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    quarterly = pd.read_excel('gdplev.xls',header=5, skiprows=[6,7],parse_cols="E,G")
    quarterly = quarterly[quarterly.index.str.startswith('20')]
    start = quarterly.index.get_loc(get_recession_start())
    end = quarterly.index.get_loc(get_recession_end())
    return quarterly.iloc[start:end,0].idxmin()
get_recession_bottom()

'2009q2'

In [17]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    data = pd.read_csv('City_Zhvi_AllHomes.csv', sep=",", header=0, index_col=[2,1])
    data = data.loc[:,data.columns.str.startswith('20')]
    data.columns = pd.to_datetime(data.columns).to_period('M')
    data = data.resample('Q',axis=1).mean()
    for col in data.columns:
        data.rename(columns={col:'{}q{}'.format(col.year, col.quarter)}, inplace=True)
    data.rename(index=states,inplace=True)   
    return data

#type(convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"])
convert_housing_data_to_quarters().info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10730 entries, (New York, New York) to (Wisconsin, Holland)
Data columns (total 67 columns):
2000q1    9355 non-null float64
2000q2    9357 non-null float64
2000q3    9374 non-null float64
2000q4    9376 non-null float64
2001q1    9409 non-null float64
2001q2    9410 non-null float64
2001q3    9420 non-null float64
2001q4    9420 non-null float64
2002q1    9435 non-null float64
2002q2    9436 non-null float64
2002q3    9454 non-null float64
2002q4    9455 non-null float64
2003q1    9509 non-null float64
2003q2    9509 non-null float64
2003q3    9568 non-null float64
2003q4    9568 non-null float64
2004q1    9623 non-null float64
2004q2    9625 non-null float64
2004q3    9666 non-null float64
2004q4    9673 non-null float64
2005q1    9730 non-null float64
2005q2    9739 non-null float64
2005q3    9772 non-null float64
2005q4    9778 non-null float64
2006q1    9807 non-null float64
2006q2    9807 non-null float64
2006q3    9818 non-null f

In [15]:
convert_housing_data_to_quarters().sample(10)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,Town of Addison,42700.000000,42033.333333,42566.666667,45133.333333,46933.333333,46800.000000,46366.666667,46200.000000,45500.000000,45233.333333,...,71400.000000,71766.666667,71733.333333,71900.000000,77233.333333,80766.666667,83000.000000,82400.000000,80066.666667,78500.0
Maryland,Ridge,149500.000000,150233.333333,160733.333333,169200.000000,171166.666667,172666.666667,168233.333333,165533.333333,167366.666667,173433.333333,...,250400.000000,267866.666667,271500.000000,272066.666667,266466.666667,267533.333333,273600.000000,273300.000000,269033.333333,258450.0
Ohio,Magnolia,95433.333333,96900.000000,97533.333333,96500.000000,96600.000000,98100.000000,101233.333333,103866.666667,105433.333333,105666.666667,...,109300.000000,111166.666667,113133.333333,113566.666667,116266.666667,118433.333333,121566.666667,122433.333333,122400.000000,122800.0
Tennessee,Medon,61633.333333,59766.666667,58433.333333,58266.666667,58566.666667,60533.333333,62033.333333,62566.666667,61800.000000,62566.666667,...,72200.000000,71733.333333,70533.333333,71400.000000,74233.333333,75533.333333,77100.000000,77933.333333,77000.000000,75950.0
Georgia,Helen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,130066.666667,130900.000000,138066.666667,138033.333333,137400.000000,136166.666667,135233.333333,135600.000000,146500.000000,148850.0
Alabama,Creola,77066.666667,78233.333333,78800.000000,81233.333333,83000.000000,82066.666667,81600.000000,82000.000000,82133.333333,83833.333333,...,115366.666667,116466.666667,116433.333333,116866.666667,119100.000000,122633.333333,123066.666667,120300.000000,118133.333333,117200.0
Vermont,Proctor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,136933.333333,139600.000000,137766.666667,135500.000000,133500.000000,133466.666667,133033.333333,134233.333333,132466.666667,129650.0
Florida,Fernandina Beach,166800.000000,169500.000000,175566.666667,181566.666667,185700.000000,192033.333333,196233.333333,198166.666667,202766.666667,204733.333333,...,268133.333333,270366.666667,274666.666667,282300.000000,286100.000000,290766.666667,294200.000000,296466.666667,300066.666667,302350.0
Indiana,Richmond,70733.333333,71500.000000,72000.000000,72666.666667,74833.333333,78466.666667,78466.666667,77800.000000,79133.333333,79900.000000,...,69433.333333,68933.333333,67900.000000,68033.333333,68100.000000,69166.666667,70766.666667,70766.666667,70366.666667,70500.0


In [8]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # General approach
    # https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/F6mWJ7SbEeeKBBKJgknU5g/replies/BugGeLcIEee5Ew6CaivGLg
    
    # How to separate two groups
    # https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/n6epwLCKEeewNAofllqCYg/replies/MXQdwbCyEeeu0xJuJy4ScA
    
    # Create new data showing the decline or growth of housing prices between the recession start and the recession bottom. 
    start = convert_housing_data_to_quarters().columns.get_loc(get_recession_start())
    bottom = convert_housing_data_to_quarters().columns.get_loc(get_recession_bottom())
    prices = convert_housing_data_to_quarters().iloc[:,start-1:bottom+1]
    prices['Price Ratio'] = prices.iloc[:,0]/prices.iloc[:,-1]

    # Divide the 'prices' into 2 sets: university towns and non-university towns
    uni_index = list(get_list_of_university_towns().itertuples(index=False, name=None))
    uni_index = [x for x in uni_index if x in prices.index]
    uni_prices = prices.loc[uni_index]
    non_prices = prices.loc[-prices.index.isin(uni_index)]
    # Or non_prices = prices.loc[list(set([x for x in prices.index if x not in uni_index]))]
    # len(non_prices)+len(uni_prices)

    print('University Town Prices')
    print('-'*40)
    print(uni_prices.iloc[:,[0,-2,-1]].info())
    print('-'*70)
    print(uni_prices.iloc[:,[0,-2,-1]].sample(10))
    print('-'*70)
    print('Non-university Town Prices')
    print('-'*40)
    print(non_prices.iloc[:,[0,-2,-1]].info())
    print('-'*70)
    print(non_prices.iloc[:,[0,-2,-1]].sample(10))
    # Run a ttest comparing the university town values to the non-university towns values, 
    # return whether the alternative hypothesis (that the two groups are the same) 
    # is true or not as well as the p-value of the confidence. 

    _,pvalue=ttest_ind(uni_prices['Price Ratio'],non_prices['Price Ratio'],nan_policy='omit',equal_var = False)
    different = pvalue<0.01
    tmp = uni_prices['Price Ratio'].mean()>non_prices['Price Ratio'].mean()
    better = 'university town'*(1-tmp)+'non-university town'*tmp
    
    return (different,pvalue,better)
run_ttest()

University Town Prices
----------------------------------------
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 269 entries, (Alabama, Montevallo) to (Wisconsin, Waukesha)
Data columns (total 3 columns):
2008q2         257 non-null float64
2009q2         259 non-null float64
Price Ratio    257 non-null float64
dtypes: float64(3)
memory usage: 72.1+ KB
None
----------------------------------------------------------------------
                                   2008q2         2009q2  Price Ratio
State          RegionName                                            
Indiana        Upland        99266.666667  101266.666667     0.980250
Florida        Orlando      181733.333333  144200.000000     1.260287
Delaware       Dover        181033.333333  171166.666667     1.057644
North Carolina Greenville   115600.000000  118066.666667     0.979108
Florida        Tallahassee  180566.666667  169700.000000     1.064035
California     Pomona       338200.000000  225666.666667     1.498671
Massachu

/opt/conda/lib/python3.6/site-packages/ipykernel/ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:32: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


(True, 0.00041476360323563257, 'university town')